In [ ]:
import os
import re
import datetime
from pathlib import Path
from dotenv import load_dotenv
from google import genai
import json

In [ ]:
def read_md(path: Path) -> str:
    try:
        with path.open("r", encoding="utf-8") as f:
            return f.read()
    except (FileNotFoundError, PermissionError, UnicodeDecodeError) as e:
        print(f"{path}の読み込みに失敗しました。{e}")

In [ ]:
# APIKEYの読み込み
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Markdownファイルの読み込み
p = Path(".") / "file.md"
md_text = read_md(p)
lines = md_text.splitlines()

# 書籍のプロティの作成
property = {}
for l in lines:
    if 'title' in l:
        title = l.split(':')[1]
    if 'author' in l:
        author = l.split(':')[1]

property['title'] = title
property['author'] = author
property['tags'] = ['kindle', 'quize']

frontmatter = '---\n'
for v in property:
    if type(property[v]) == list:
        # frontmatter += v + ': [' + ','.join(property['tags']) + ']\n'
        frontmatter += v + ': ' + json.dumps(property[v]) + '\n'
    if type(property[v]) == str:
        frontmatter += v + ': ' + property[v] + '\n'
frontmatter += '---\n'
print(frontmatter)

In [ ]:
print(md_text)

In [ ]:
# Highlight箇所の抽出
highlightAfterPattern = r"##\s*Highlights\n*(.*)—\slocation"
dashWrappedPattern = r"---\n*(.*)\n*—\slocation"
matches = re.findall(highlightAfterPattern, md_text)
matches += re.findall(dashWrappedPattern, md_text)
# print(matches)

highlightMarkdown = ''
for text in matches:
    highlightMarkdown += '- ' + text + '\n'
print(highlightMarkdown)


In [ ]:
# mdファイルの要約
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents= '以下は書籍の内容で重点的に復習したい箇所を' \
        'ハイライトした箇所をMarkdown形式にまとめたものです。要約してください。' \
        + highlightMarkdown)
print(response.text)